In [ ]:
import branca
import dask.distributed
import folium
import folium.plugins
import geopandas as gpd
import numpy as np
import rasterio as rio
import rasterio.features
import shapely.geometry
import xrspatial.multispectral as ms
from branca.element import Element, Figure
from IPython.display import HTML, display
from odc.stac import configure_rio, stac_load
from pystac_client import Client

from utils import convert_bounds, image_on_map

In [ ]:
# stack configuration
cfg = {
    "sentinel-s2-l2a-cogs": {
        "assets": {
            "*": {"data_type": "uint16", "nodata": 0},
            "SCL": {"data_type": "uint8", "nodata": 0},
            "visual": {"data_type": "uint8", "nodata": 0},
        },
        "aliases": {"red": "B04", "green": "B03", "blue": "B02"},
    },
    "*": {"warnings": "ignore"},
}

In [ ]:
# start the dask cluster
client = dask.distributed.Client()
configure_rio(cloud_defaults=True, aws={"aws_unsigned": True}, client=client)
client

# Select a bounding box for your query
Use the rectangle tool and then click on it to get the recatngle json

In [ ]:
fig = branca.element.Figure(width="600px", height="600px")
m = folium.Map()
fig.add_child(m)
draw = folium.plugins.Draw(
    position="topleft",
    draw_options={"polyline": {"allowIntersection": False}},
    edit_options={"poly": {"allowIntersection": False}},
).add_to(m)
fig

In [ ]:
# paste here the bbox json
aoi = {
    "type": "Polygon",
    "coordinates": [
        [
            [-61.259766, -64.54844],
            [-61.259766, -61.648162],
            [-53.085938, -61.648162],
            [-53.085938, -64.54844],
            [-61.259766, -64.54844],
        ]
    ],
}
bbox = rasterio.features.bounds(aoi)

In [ ]:
catalog = Client.open("https://earth-search.aws.element84.com/v0")

query = catalog.search(
    collections=["sentinel-s2-l2a-cogs"],
    datetime="2022-5-1/2022-9-21",  # from / to
    limit=100,
    bbox=bbox,
    query={"eo:cloud_cover": {"lt": 0.1}},
)

items = list(query.get_items())
print(f"Found: {len(items)} datasets")

# Convert STAC items into a GeoJSON FeatureCollection
stac_json = query.get_all_items_as_dict()

Show on a map the foodprints of the found datasets 

In [ ]:
# https://github.com/python-visualization/folium/issues/1501
gdf = gpd.GeoDataFrame.from_features(stac_json, "epsg:4326")

# Compute granule id from components
gdf["granule"] = (
    gdf["sentinel:utm_zone"].apply(lambda x: f"{x:02d}")
    + gdf["sentinel:latitude_band"]
    + gdf["sentinel:grid_square"]
)

fig = Figure(width="600px", height="600px")
map1 = folium.Map()
fig.add_child(map1)

folium.GeoJson(
    shapely.geometry.box(*bbox),
    style_function=lambda x: dict(fill=False, weight=1, opacity=0.7, color="olive"),
    name="Query",
).add_to(map1)

gdf.explore(
    "granule",
    categorical=True,
    tooltip=[
        "granule",
        "datetime",
        "sentinel:data_coverage",
        "eo:cloud_cover",
    ],
    popup=True,
    style_kwds=dict(fillOpacity=0.1, width=2),
    name="STAC",
    m=map1,
)

map1.fit_bounds(bounds=convert_bounds(gdf.unary_union.bounds))
folium.LayerControl().add_to(map1)

display(fig)

In [ ]:
it = [i for i in items if "20EPQ" in i.id]

In [ ]:
it[1]

In [ ]:
scene = stac_load(
    [it[1]],
    bands=["B04", "B03", "B02"],
    crs="epsg:3857",
    resolution=50,
    chunks={}
)

In [ ]:
scene = scene.where(lambda x: x > 0, other=np.nan).to_array("band").squeeze("time").chunk("auto")

In [ ]:
scene

In [ ]:
image_on_map(ms.true_color(*scene).compute(), bbox)

In [ ]:
# Since we will plot it on a map we need to use `EPSG:3857` projection
crs = "epsg:3857"

data = stac_load(
    items,
    bands=["B04", "B03", "B02"],
    crs=crs,
    resolution=10,
    chunks={},  # <-- use Dask
    # stac_cfg=cfg,
    # bbox=bbox,
)

data

In [ ]:
data = (
    data.where(lambda x: x > 0, other=np.nan).to_array("band").chunk("auto")
)  # sentinel-2 uses 0 as nodata
data

Inspect one of the scenes on the map

In [ ]:
# use ms.true_color to convert RGB chanels and improve the color
selection = data.isel(time=1)

In [ ]:
selection

In [ ]:
image_on_map(ms.true_color(*selection).compute(), bbox)
m

In [ ]:
ms.true_color(*selection).plot.imshow()

Compute the cloudless mossaic by taking the median of each pixel across time

In [ ]:
%%time
median = data.median(dim="time").compute()

In [ ]:
image = ms.true_color(*median)  # expects red, green, blue DataArrays

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 12))

ax.set_axis_off()
image.plot.imshow(ax=ax);

In [ ]:
fig = Figure(width="600px", height="600px")
m = folium.Map()
fig.add_child(m)
image.odc.add_to(m)

m.fit_bounds(bounds=convert_bounds(gdf.unary_union.bounds))
folium.LayerControl().add_to(m)
m